# Homework and bake-off: Relation extraction using distant supervision

In [1]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [2]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

In [3]:
# Set all the random seeds for reproducibility. Only the
# system seed is relevant for this notebook.

utils.fix_random_seeds()

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [4]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [5]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [6]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [7]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [8]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [9]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [10]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [11]:
featurizers = [simple_bag_of_words_featurizer]

In [12]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [13]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.893      0.394      0.713        340       5716
author                    0.780      0.536      0.715        509       5885
capital                   0.583      0.221      0.439         95       5471
contains                  0.795      0.589      0.743       3904       9280
film_performance          0.783      0.560      0.725        766       6142
founders                  0.819      0.392      0.672        380       5756
genre                     0.469      0.176      0.352        170       5546
has_sibling               0.890      0.242      0.580        499       5875
has_spouse                0.895      0.343      0.677        594       5970
is_a                      0.665      0.215      0.469        497       5873
nationality               0.624      0.176      0.413        301       5677
parents     

Studying model weights might yield insights:

In [14]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.538 Córdoba
     2.390 Valais
     2.389 Taluks
     ..... .....
    -1.295 Australia
    -1.469 Europe
    -1.860 America

Highest and lowest feature weights for relation author:

     3.062 wrote
     2.771 books
     2.448 author
     ..... .....
    -1.927 directed
    -1.934 same
    -2.101 or

Highest and lowest feature weights for relation capital:

     3.606 capital
     2.073 city
     1.819 town
     ..... .....
    -1.172 and
    -1.286 also
    -1.611 Antrim

Highest and lowest feature weights for relation contains:

     2.892 third-largest
     2.396 within
     2.177 transferred
     ..... .....
    -2.480 Henley-on-Thames
    -2.850 2002
    -4.009 Antrim

Highest and lowest feature weights for relation film_performance:

     3.814 co-starring
     3.761 starring
     3.532 opposite
     ..... .....
    -1.990 Zigzag
    -2.120 Tamil
    -2.141 Iruvar

Highest and lowest feature weights for relation foun

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [15]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [16]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [17]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [18]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.864      0.447      0.728        340       5716
author                    0.815      0.434      0.694        509       5885
capital                   0.625      0.211      0.448         95       5471
contains                  0.654      0.417      0.587       3904       9280
film_performance          0.822      0.325      0.629        766       6142
founders                  0.740      0.239      0.522        380       5756
genre                     0.333      0.065      0.182        170       5546
has_sibling               0.902      0.240      0.582        499       5875
has_spouse                0.876      0.357      0.679        594       5970
is_a                      0.728      0.151      0.413        497       5873
nationality               0.633      0.206      0.447        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [19]:
from sklearn.svm import SVC
svm_model = lambda: SVC(kernel='linear')

In [20]:
def run_svm_model_factory():
    results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=featurizers,
        model_factory=svm_model,
        verbose=True)
    return results

In [21]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."

In [22]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.848      0.376      0.678        340       5716
author                    0.737      0.599      0.704        509       5885
capital                   0.562      0.284      0.470         95       5471
contains                  0.776      0.602      0.734       3904       9280
film_performance          0.752      0.624      0.722        766       6142
founders                  0.720      0.426      0.633        380       5756
genre                     0.523      0.265      0.438        170       5546
has_sibling               0.846      0.230      0.551        499       5875
has_spouse                0.873      0.347      0.670        594       5970
is_a                      0.616      0.278      0.495        497       5873
nationality               0.495      0.179      0.366        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [23]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    # Append these to the end of the keys you add/access in
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word + object_subject_suffix] += 1

    return feature_counter


# Call to `rel_ext.experiment`:
directional_bag_of_words_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[directional_bag_of_words_featurizer],
        verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.899      0.418      0.730        340       5716
author                    0.807      0.599      0.755        509       5885
capital                   0.730      0.284      0.556         95       5471
contains                  0.812      0.670      0.779       3904       9280
film_performance          0.840      0.653      0.795        766       6142
founders                  0.824      0.405      0.683        380       5756
genre                     0.618      0.276      0.496        170       5546
has_sibling               0.871      0.244      0.576        499       5875
has_spouse                0.903      0.359      0.692        594       5970
is_a                      0.719      0.247      0.521        497       5873
nationality               0.694      0.226      0.491        301       5677
parents     

In [24]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [25]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

In [26]:
# How many feature names does the vectorizer have for the experiment run in the previous step? 40537
directional_bag_of_words_vectorizer = directional_bag_of_words_results['vectorizer']
directional_bag_of_words_features_names = directional_bag_of_words_vectorizer.get_feature_names()
print('number of feature names: {}'.format(len(directional_bag_of_words_features_names)))

number of feature names: 40756


### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [27]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for bigram in get_tag_bigrams(ex.middle_POS):
            feature_counter[bigram] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for bigram in get_tag_bigrams(ex.middle_POS):
            feature_counter[bigram] += 1

    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"
    
    tags = [start_symbol] + get_tags(s) + [end_symbol]
    return get_bigrams(tags)


def get_bigrams(tags):
    """Given a list of POS sequences, returns a list of bigram POS sequences"""
    return [t[0] + ' ' + t[1] for t in zip(tags[:-1], tags[1:])]


def get_tags(s):
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)

# Call to `rel_ext.experiment`:
middle_bigram_pos_tag_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[middle_bigram_pos_tag_featurizer],
        verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.823      0.356      0.652        340       5716
author                    0.712      0.340      0.584        509       5885
capital                   0.457      0.168      0.340         95       5471
contains                  0.749      0.585      0.709       3904       9280
film_performance          0.693      0.448      0.625        766       6142
founders                  0.533      0.168      0.372        380       5756
genre                     0.525      0.182      0.382        170       5546
has_sibling               0.675      0.166      0.419        499       5875
has_spouse                0.778      0.259      0.556        594       5970
is_a                      0.549      0.157      0.366        497       5873
nationality               0.413      0.086      0.235        301       5677
parents     

In [28]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [29]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [30]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for synset in get_synsets(ex.middle_POS):
            feature_counter[synset] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for synset in get_synsets(ex.middle_POS):
            feature_counter[synset] += 1

    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets
    associated with elements of `s`.
    """
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]

    return [str(synset) for p in wt for synset in wn.synsets(p[0], convert_tag(p[1]))]


def convert_tag(t):
    """Converts tags so that they can be used by WordNet:

    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None


# Call to `rel_ext.experiment`:
synset_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[synset_featurizer],
        verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.827      0.324      0.631        340       5716
author                    0.790      0.444      0.684        509       5885
capital                   0.657      0.242      0.489         95       5471
contains                  0.779      0.581      0.729       3904       9280
film_performance          0.755      0.552      0.704        766       6142
founders                  0.753      0.376      0.627        380       5756
genre                     0.410      0.200      0.339        170       5546
has_sibling               0.820      0.218      0.529        499       5875
has_spouse                0.838      0.305      0.621        594       5970
is_a                      0.613      0.223      0.455        497       5873
nationality               0.523      0.150      0.349        301       5677
parents     

In [31]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [32]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

In [33]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams

In [34]:
def get_ngrams(tokens, n=1):
    if n == 1:
        return [' '.join(ngram) for ngram in ngrams(tokens, n)]
    else:
        return [' '.join(ngram) for ngram 
                in ngrams(tokens, n, pad_left=True, pad_right=True, 
                          left_pad_symbol='<s>', right_pad_symbol='</s>')]

In [35]:
def directional_middle_unigram_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for ngram in get_ngrams(ex.middle.split(' '), 1):
            feature_counter[ngram + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for ngram in get_ngrams(ex.middle.split(' '), 1):
            feature_counter[ngram + object_subject_suffix] += 1

    return feature_counter

In [36]:
def directional_middle_bigram_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for ngram in get_ngrams(ex.middle.split(' '), 2):
            feature_counter[ngram + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for ngram in get_ngrams(ex.middle.split(' '), 2):
            feature_counter[ngram + object_subject_suffix] += 1

    return feature_counter

In [37]:
def directional_middle_trigram_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for ngram in get_ngrams(ex.middle.split(' '), 3):
            feature_counter[ngram + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for ngram in get_ngrams(ex.middle.split(' '), 3):
            feature_counter[ngram + object_subject_suffix] += 1

    return feature_counter

In [38]:
def directional_middle_pos_unigram_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for ngram in get_ngrams(get_tags(ex.middle_POS), 1):
            feature_counter[ngram + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for ngram in get_ngrams(get_tags(ex.middle_POS), 1):
            feature_counter[ngram + object_subject_suffix] += 1

    return feature_counter

In [39]:
def directional_middle_pos_bigram_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for ngram in get_ngrams(get_tags(ex.middle_POS), 2):
            feature_counter[ngram + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for ngram in get_ngrams(get_tags(ex.middle_POS), 2):
            feature_counter[ngram + object_subject_suffix] += 1

    return feature_counter

In [40]:
def directional_middle_pos_trigram_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for ngram in get_ngrams(get_tags(ex.middle_POS), 3):
            feature_counter[ngram + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for ngram in get_ngrams(get_tags(ex.middle_POS), 3):
            feature_counter[ngram + object_subject_suffix] += 1

    return feature_counter

In [42]:
def detailed_entity_mention_pos_featurizer(kbt, corpus, feature_counter):
    first_entity_subject_object_suffix = "_1_SO"
    second_entity_subject_object_suffix = "_2_SO"
    first_entity_object_subject_suffix = "_1_OS"
    second_entity_object_subject_suffix = "_2_OS"
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        feature_counter[' '.join(get_tags(ex.mention_1_POS)) + first_entity_subject_object_suffix] += 1
        feature_counter[' '.join(get_tags(ex.mention_2_POS)) + second_entity_subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter[' '.join(get_tags(ex.mention_1_POS)) + first_entity_object_subject_suffix] += 1
        feature_counter[' '.join(get_tags(ex.mention_2_POS)) + second_entity_object_subject_suffix] += 1
            
    return feature_counter

/opt/miniconda3/envs/nlu/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.868      0.291      0.622        340       5716
author                    0.842      0.409      0.695        509       5885
capital                   0.450      0.095      0.257         95       5471
contains                  0.715      0.428      0.630       3904       9280
film_performance          0.780      0.522      0.710        766       6142
founders                  0.724      0.166      0.433        380       5756
genre                     0.750      0.335      0.601        170       5546
has_sibling               0.488      0.040      0.151        499       5875
has_spouse                0.775      0.133      0.394        594       5970
is_a                      0.594      0.185      0.412        497       5873
nationality               0.652      0.100      0.309        301       5677
parents     

In [43]:
_ = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[directional_middle_unigram_featurizer, 
                     directional_middle_bigram_featurizer, 
                     directional_middle_trigram_featurizer,
                     directional_middle_pos_unigram_featurizer, 
                     directional_middle_pos_bigram_featurizer,
                     directional_middle_pos_trigram_featurizer,
                     detailed_entity_mention_pos_featurizer],
        verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.870      0.353      0.673        340       5716
author                    0.868      0.682      0.823        509       5885
capital                   0.703      0.274      0.535         95       5471
contains                  0.854      0.746      0.830       3904       9280
film_performance          0.890      0.789      0.867        766       6142
founders                  0.818      0.521      0.734        380       5756
genre                     0.803      0.624      0.759        170       5546
has_sibling               0.873      0.275      0.608        499       5875
has_spouse                0.876      0.367      0.686        594       5970
is_a                      0.744      0.521      0.686        497       5873
nationality               0.738      0.392      0.627        301       5677
parents     

In [50]:
def middle_len_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    middle_len_prefix = "middle_len"
    
    middle_len_so = 0
    ex_num_so = 0
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        middle_len_so += len(ex.middle)
        ex_num_so += 1
    feature_counter[middle_len_prefix + subject_object_suffix] = 0
    if ex_num_so > 0:
        feature_counter[middle_len_prefix + subject_object_suffix] = middle_len_so / ex_num_so
    
    middle_len_os = 0
    ex_num_os = 0
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        middle_len_os += len(ex.middle)
        ex_num_os += 1
    feature_counter[middle_len_prefix + object_subject_suffix] = 0
    if ex_num_os > 0:
        feature_counter[middle_len_prefix + object_subject_suffix] = middle_len_os / ex_num_os

    return feature_counter

In [51]:
_ = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[directional_middle_unigram_featurizer, 
                     directional_middle_bigram_featurizer, 
                     directional_middle_trigram_featurizer,
                     directional_middle_pos_unigram_featurizer, 
                     directional_middle_pos_bigram_featurizer,
                     directional_middle_pos_trigram_featurizer,
                     detailed_entity_mention_pos_featurizer,
                     middle_len_featurizer],
        verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.860      0.415      0.708        340       5716
author                    0.888      0.829      0.876        509       5885
capital                   0.717      0.347      0.591         95       5471
contains                  0.871      0.819      0.860       3904       9280
film_performance          0.883      0.787      0.862        766       6142
founders                  0.818      0.521      0.734        380       5756
genre                     0.786      0.606      0.742        170       5546
has_sibling               0.831      0.287      0.602        499       5875
has_spouse                0.819      0.389      0.671        594       5970
is_a                      0.740      0.509      0.678        497       5873
nationality               0.835      0.754      0.817        301       5677
parents     

In [71]:
def directional_left_window_1_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    pad = '#PAD#'
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        feature_counter[([pad] + ex.left.split(' ')[-1:])[-1] + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter[([pad] + ex.left.split(' ')[-1:])[-1] + object_subject_suffix] += 1
            
    return feature_counter

In [88]:
def detailed_left_right_window_1_featurizer(kbt, corpus, feature_counter):
    left_subject_object_suffix = "_L_SO"
    left_object_subject_suffix = "_L_OS"
    right_subject_object_suffix = "_R_SO"
    right_object_subject_suffix = "_R_OS"
    pad = '#PAD#'
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        feature_counter[([pad] + ex.left.split(' ')[-1:])[-1] + left_subject_object_suffix] += 1
        feature_counter[(ex.right.split(' ')[0:1] + [pad])[0] + right_subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter[([pad] + ex.left.split(' ')[-1:])[-1] + left_object_subject_suffix] += 1
        feature_counter[(ex.right.split(' ')[0:1] + [pad])[0] + right_object_subject_suffix] += 1
            
    return feature_counter

In [72]:
_ = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[directional_middle_unigram_featurizer, 
                     directional_middle_bigram_featurizer, 
                     directional_middle_trigram_featurizer,
                     directional_middle_pos_unigram_featurizer, 
                     directional_middle_pos_bigram_featurizer,
                     directional_middle_pos_trigram_featurizer,
                     detailed_entity_mention_pos_featurizer,
                     middle_len_featurizer,
                     directional_left_window_1_featurizer],
        verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.847      0.441      0.716        340       5716
author                    0.884      0.835      0.873        509       5885
capital                   0.680      0.358      0.576         95       5471
contains                  0.885      0.842      0.876       3904       9280
film_performance          0.885      0.800      0.866        766       6142
founders                  0.842      0.671      0.801        380       5756
genre                     0.800      0.612      0.754        170       5546
has_sibling               0.845      0.351      0.659        499       5875
has_spouse                0.821      0.402      0.680        594       5970
is_a                      0.801      0.706      0.780        497       5873
nationality               0.831      0.751      0.814        301       5677
parents     

In [95]:
def detailed_left_right_window_2_featurizer(kbt, corpus, feature_counter):
    left_subject_object_suffix = "_L_SO"
    left_object_subject_suffix = "_L_OS"
    right_subject_object_suffix = "_R_SO"
    right_object_subject_suffix = "_R_OS"
    pad = '#PAD#'
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        feature_counter[' '.join(([pad] * 2 + ex.left.split(' ')[-2:])[-2:]) + left_subject_object_suffix] += 1
        feature_counter[' '.join((ex.right.split(' ')[0:2] + [pad] * 2)[0:2]) + right_subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter[' '.join(([pad] * 2 + ex.left.split(' ')[-2:])[-2:]) + left_object_subject_suffix] += 1
        feature_counter[' '.join((ex.right.split(' ')[0:2] + [pad] * 2)[0:2]) + right_object_subject_suffix] += 1
            
    return feature_counter

In [98]:
_ = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[directional_middle_unigram_featurizer, 
                     directional_middle_bigram_featurizer, 
                     directional_middle_trigram_featurizer,
                     directional_middle_pos_unigram_featurizer, 
                     directional_middle_pos_bigram_featurizer,
                     directional_middle_pos_trigram_featurizer,
                     detailed_entity_mention_pos_featurizer,
                     middle_len_featurizer,
                     detailed_left_right_window_1_featurizer,
                     detailed_left_right_window_2_featurizer],
        verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.876      0.456      0.740        340       5716
author                    0.879      0.839      0.870        509       5885
capital                   0.712      0.389      0.611         95       5471
contains                  0.901      0.855      0.892       3904       9280
film_performance          0.889      0.816      0.873        766       6142
founders                  0.838      0.668      0.798        380       5756
genre                     0.828      0.653      0.786        170       5546
has_sibling               0.864      0.357      0.673        499       5875
has_spouse                0.806      0.428      0.685        594       5970
is_a                      0.808      0.728      0.791        497       5873
nationality               0.830      0.761      0.815        301       5677
parents     

In [103]:
def detailed_left_right_pos_window_1_featurizer(kbt, corpus, feature_counter):
    left_subject_object_suffix = "_L_SO"
    left_object_subject_suffix = "_L_OS"
    right_subject_object_suffix = "_R_SO"
    right_object_subject_suffix = "_R_OS"
    pad = '#PAD#'
        
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        feature_counter[([pad] + get_tags(ex.left_POS)[-1:])[-1] + left_subject_object_suffix] += 1
        feature_counter[(get_tags(ex.right_POS)[0:1] + [pad])[0] + right_subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter[([pad] + get_tags(ex.left_POS)[-1:])[-1] + left_object_subject_suffix] += 1
        feature_counter[(get_tags(ex.right_POS)[0:1] + [pad])[0] + right_object_subject_suffix] += 1
            
    return feature_counter

In [105]:
def detailed_left_right_pos_window_2_featurizer(kbt, corpus, feature_counter):
    left_subject_object_suffix = "_L_SO"
    left_object_subject_suffix = "_L_OS"
    right_subject_object_suffix = "_R_SO"
    right_object_subject_suffix = "_R_OS"
    pad = '#PAD#'
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        feature_counter[' '.join(([pad] * 2 + get_tags(ex.left_POS)[-2:])[-2:]) + left_subject_object_suffix] += 1
        feature_counter[' '.join((get_tags(ex.right_POS)[0:2] + [pad] * 2)[0:2]) + right_subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter[' '.join(([pad] * 2 + get_tags(ex.left_POS)[-2:])[-2:]) + left_object_subject_suffix] += 1
        feature_counter[' '.join((get_tags(ex.right_POS)[0:2] + [pad] * 2)[0:2]) + right_object_subject_suffix] += 1
            
    return feature_counter

In [106]:
_ = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[directional_middle_unigram_featurizer, 
                     directional_middle_bigram_featurizer, 
                     directional_middle_trigram_featurizer,
                     directional_middle_pos_unigram_featurizer, 
                     directional_middle_pos_bigram_featurizer,
                     directional_middle_pos_trigram_featurizer,
                     detailed_entity_mention_pos_featurizer,
                     middle_len_featurizer,
                     detailed_left_right_window_1_featurizer,
                     detailed_left_right_window_2_featurizer,
                     detailed_left_right_pos_window_1_featurizer,
                     detailed_left_right_pos_window_2_featurizer],
        verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.863      0.465      0.737        340       5716
author                    0.882      0.839      0.873        509       5885
capital                   0.720      0.379      0.610         95       5471
contains                  0.902      0.854      0.892       3904       9280
film_performance          0.888      0.821      0.874        766       6142
founders                  0.830      0.692      0.798        380       5756
genre                     0.820      0.671      0.785        170       5546
has_sibling               0.814      0.387      0.667        499       5875
has_spouse                0.759      0.439      0.662        594       5970
is_a                      0.804      0.724      0.786        497       5873
nationality               0.855      0.781      0.839        301       5677
parents     

In [111]:
# from sklearn.ensemble import RandomForestClassifier

# rf = lambda: RandomForestClassifier(n_jobs=-1, random_state=42)

# _ = rel_ext.experiment(
#         splits,
#         train_split='train',
#         test_split='dev',
#         featurizers=[directional_middle_unigram_featurizer, 
#                      directional_middle_bigram_featurizer, 
#                      directional_middle_trigram_featurizer,
#                      directional_middle_pos_unigram_featurizer, 
#                      directional_middle_pos_bigram_featurizer,
#                      directional_middle_pos_trigram_featurizer,
#                      detailed_entity_mention_pos_featurizer,
#                      middle_len_featurizer,
#                      detailed_left_right_window_1_featurizer,
#                      detailed_left_right_window_2_featurizer],
#         model_factory=rf,
#         verbose=True)

# relation              precision     recall    f-score    support       size
# ------------------    ---------  ---------  ---------  ---------  ---------
# adjoins                   0.952      0.524      0.818        340       5716
# author                    0.882      0.778      0.859        509       5885
# capital                   0.892      0.347      0.679         95       5471
# contains                  0.907      0.825      0.889       3904       9280
# film_performance          0.922      0.691      0.864        766       6142
# founders                  0.908      0.492      0.777        380       5756
# genre                     0.871      0.318      0.646        170       5546
# has_sibling               0.917      0.531      0.801        499       5875
# has_spouse                0.952      0.606      0.855        594       5970
# is_a                      0.919      0.499      0.786        497       5873
# nationality               0.925      0.575      0.825        301       5677
# parents                   0.961      0.638      0.873        312       5688
# place_of_birth            0.931      0.403      0.738        233       5609
# place_of_death            0.939      0.289      0.648        159       5535
# profession                0.939      0.502      0.800        247       5623
# worked_at                 0.923      0.347      0.693        242       5618
# ------------------    ---------  ---------  ---------  ---------  ---------
# macro-average             0.921      0.523      0.784       9248      95264

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.952      0.524      0.818        340       5716
author                    0.882      0.778      0.859        509       5885
capital                   0.892      0.347      0.679         95       5471
contains                  0.907      0.825      0.889       3904       9280
film_performance          0.922      0.691      0.864        766       6142
founders                  0.908      0.492      0.777        380       5756
genre                     0.871      0.318      0.646        170       5546
has_sibling               0.917      0.531      0.801        499       5875
has_spouse                0.952      0.606      0.855        594       5970
is_a                      0.919      0.499      0.786        497       5873
nationality               0.925      0.575      0.825        301       5677
parents     

In [116]:
from sklearn.ensemble import RandomForestClassifier

rf = lambda: RandomForestClassifier(n_jobs=-1, random_state=42)

_ = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[directional_middle_unigram_featurizer, 
                     directional_middle_bigram_featurizer, 
                     directional_middle_trigram_featurizer,
                     directional_middle_pos_unigram_featurizer, 
                     directional_middle_pos_bigram_featurizer,
                     directional_middle_pos_trigram_featurizer,
                     detailed_entity_mention_pos_featurizer,
                     middle_len_featurizer,
                     detailed_left_right_window_1_featurizer,
                     detailed_left_right_window_2_featurizer,
                     detailed_left_right_pos_window_1_featurizer,
                     detailed_left_right_pos_window_2_featurizer],
        model_factory=rf,
        verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.967      0.524      0.827        340       5716
author                    0.900      0.780      0.873        509       5885
capital                   0.919      0.358      0.700         95       5471
contains                  0.913      0.827      0.894       3904       9280
film_performance          0.925      0.689      0.866        766       6142
founders                  0.910      0.476      0.770        380       5756
genre                     0.889      0.282      0.622        170       5546
has_sibling               0.957      0.531      0.825        499       5875
has_spouse                0.949      0.598      0.849        594       5970
is_a                      0.931      0.491      0.790        497       5873
nationality               0.966      0.561      0.844        301       5677
parents     

In [ ]:
from nltk.parse.stanford import StanfordDependencyParser

path_to_jar = os.path.join('stanford-parser-4.0.0', 'stanford-parser.jar')
path_to_models_jar = os.path.join('stanford-parser-4.0.0', 'stanford-parser-4.0.0-models.jar')

dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)

sentence = 'my name is andrew.'
result = list(dependency_parser.raw_parse(sentence))[0]
[item for item in result.triples()]

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [44]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# IMPORT ANY MODULES BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: MY_NUMBER
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass

# STOP COMMENT: Please do not remove this comment.

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [45]:
# Enter your bake-off assessment code in this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE


In [46]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported
# by the code above. Please enter only a number between
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
